In [26]:
import cv2
from ultralytics import solutions,YOLO

In [28]:
model = YOLO("yolo11n-pose.pt")

In [30]:
source = 0

In [32]:
cap = cv2.VideoCapture(source)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

In [33]:
if not cap.isOpened():
    print("Kamera Açılmadı!")
    exit()


In [34]:
gym = solutions.AIGym(
    show=True,  # Görüntüyü göster
    kpts=[11, 13, 15],  # Sol kalça(11), sol diz(13) ve sol ayak bileği(15) noktaları
    up_angle=160.0,  # Dik duruş için daha yüksek açı
    down_angle=70.0,  # Squat pozisyonu için daha düşük açı
    model="yolo11n-pose.pt"  # Pose estimation modeli
)

Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 160.0, 'down_angle': 70.0, 'kpts': [11, 13, 15], 'analytics_type': 'line', 'json_file': None, 'show': True, 'model': 'yolo11n-pose.pt'}


In [35]:
squat_count = 0  # Squat sayaç
squat_down = False  # Kullanıcı aşağıda mı?

def calculate_angle(a, b, c):
    """İki vektör arasındaki açıyı hesapla"""
    a = np.array(a)  # Kalça
    b = np.array(b)  # Diz
    c = np.array(c)  # Ayak bileği

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


In [36]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Kare alınamadı, çıkılıyor...")
        break
    # OpenCV görüntüsü BGR formatındadır, YOLO RGB bekler. Dönüştürme yapalım.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLO modelini kare üzerinde çalıştır
    
    results = model.predict(frame_rgb, task="pose")


    for r in results:
        # Anahtar noktaları al
        keypoints = r.keypoints.xy.numpy() if r.keypoints is not None else None
        if keypoints is not None:
            try:
                # Sol bacak noktaları (kalça, diz, ayak bileği)
                hip = keypoints[11]
                knee = keypoints[13]
                ankle = keypoints[15]

                # Açıyı hesapla
                angle = calculate_angle(hip, knee, ankle)

                # Squat sayacı
                if angle < 70 and not squat_down:  # Kullanıcı aşağı inmiş
                    squat_down = True
                if angle > 160 and squat_down:  # Kullanıcı yukarı çıkmış
                    squat_count += 1
                    squat_down = False

                # Açıyı ve sayaç değerini ekrana yazdır
                cv2.putText(frame, f"Aci: {int(angle)} derece", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Squat Sayisi: {squat_count}", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            except IndexError:
                pass

    cv2.imshow("Squat Counter", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



0: 480x640 1 person, 162.5ms
Speed: 7.1ms preprocess, 162.5ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 202.5ms
Speed: 4.0ms preprocess, 202.5ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 175.0ms
Speed: 3.6ms preprocess, 175.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 158.0ms
Speed: 4.0ms preprocess, 158.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 178.1ms
Speed: 3.0ms preprocess, 178.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 171.3ms
Speed: 4.0ms preprocess, 171.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 126.2ms
Speed: 4.0ms preprocess, 126.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 145.6ms
Speed: 2.4ms preprocess, 145.6ms inference, 2.0ms postprocess per image at

In [37]:
cap.release()
cv2.destroyAllWindows()